In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import transformers
import load_data
import torch
from torch import nn
from transformers import BertModel, BertTokenizer, CamembertModel, CamembertTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
import numpy as np
import os
from training_audio_model import *
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu') # GPU ou CPU
print(torch.cuda.is_available())
# Chargement des données
transcr_path='../paco-cheese/transcr'
data=load_data.load_all_ipus(folder_path=transcr_path,load_words=True)

data[:20]

C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'load_data'

In [2]:
from utils import *
y=create_y(data)

train_size = int(len(data) * 0.8)
train_size=train_size
X_train = data[:train_size]
y_train = y[:train_size]


X_test = data[train_size:]
y_test = y[train_size:]

y_test=y[train_size:]

y_test[:20]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0]

In [3]:
def pred_y(df, x):
    y = []
    # Vérifiez que l'index i+1 est dans le range du dataframe pour éviter le KeyError
    df = df.reset_index(drop=True)
    for i in range(len(df)):
        if i + 1 in df.index:
            # Calculer le temps d'attente jusqu'à ce que la parole suivante commence
            wait_time = df.loc[i + 1, 'start_words'] - df.loc[i, 'stop_words']
            # Vérifiez si le temps d'attente est supérieur à x secondes et si le locuteur change
            if wait_time > x:
                y.append(1)  # Il y a un changement de parole et l'attente est plus longue que x secondes
            else:
                y.append(0)  # Pas de changement de locuteur significatif ou pas de changement de locuteur
        else:
            y.append(0)  # Pas de locuteur suivant à comparer, donc on ajoute 0
    return y

# Crée une liste de label en fonction d'une fenêtre temporelle
def create_y_time(df, window_time):
    y = [0] 
    for i in range(0, len(df)-1):
        j = i + 1
        stop = False
        while j < len(df) and df['start_words'][j] < df['stop_words'][i] + window_time and not stop:
            if df['speaker'][i] != df['speaker'][j]:
                y.append(1)  # Changement de locuteur
                stop = True
            j += 1
        if not stop:
            y.append(0)  # Pas de changement de locuteur dans la fenêtre de temps
    return y



y_pred=pred_y(X_test, 0.9)


In [5]:
f1 = f1_score(y_test,y_pred)#, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)
print(f'Test F1 Score: {f1}')
print(f'Confusion Matrix:\n{conf_matrix}')

# Calculer et afficher les informations spécifiques
total_class_0 = np.sum(conf_matrix[0])
total_class_1 = np.sum(conf_matrix[1])
detected_class_0 = conf_matrix[0, 0]  # Vrais positifs pour la classe 0
detected_class_1 = conf_matrix[1, 1]  # Vrais positifs pour la classe 1

print(f'Nombre d\'éléments de classe 0 détectés : {detected_class_0} sur {total_class_0}')
print(f'Nombre d\'éléments de classe 1 détectés : {detected_class_1} sur {total_class_1}')


Test F1 Score: 0.12061591103507271
Confusion Matrix:
[[19912   418]
 [ 1638   141]]
Nombre d'éléments de classe 0 détectés : 19912 sur 20330
Nombre d'éléments de classe 1 détectés : 141 sur 1779
